# Build and Execute Merge for Dimension for Snowflake (Project Setup)

This notebook contains the SQL Set-up scripts for the eltSnap project "Build and Execute Merge for Dimension for Snowflake"

These script needs to be run **after** running the notebook called **Ingest ADS Notebook Files (Project)**

## This project requires a free Snowflake ODBC driver 

> The Snowflake ODBC driver can be downloaded [here](https://docs.snowflake.com/en/user-guide/odbc.html)

## Create a DSN using the ODBC Data Source application

### 1\. Navigate to the 'System DSN' tab and click 'Add' to create a new DSN

![System DSN](.\odbc_app.png)

### 2\. Choose 'SnowflakeDSIIDriver' and click 'Finish'

![Add Data Source](.\add_odbc_data_source.png)

### 3\. Configure the connection settings

Use the following connection settings

| Property | Description |
| --- | --- |
| Data Source | Friendly name of the data source |
| User | User name to access the Snowflake database |
| Password | Password to access the Snowflake database |
| Server | Domain name of your Snowflake account |
| Database | Name of the Snowflake database |
| Schema | Name of the Snowflake schema |
| Warehouse | Name of the Snowflake warehouse |
| Role | The Snowflake role assigned to the user |
| Tracing (0-6) | Sets the amount of detail shown in the log file |
```
Tracing levels
- 0. Disable Tracing
- 1. Fatal error tracing
- 2. Error tracing
- 3. Warning tracing
- 4. Info tracing
- 5. Debug tracing
- 6. Detailed tracing
```
The following image shows the dialog box where you can input the connection settings

![Configuration Dialog](.\odbc_config.png)

> For more information a Snowflake ODBC user guide can be found [here](https://docs.snowflake.com/en/user-guide/odbc-parameters.html)

### 4\. Add password to Registry Editor

The password in the ODBC configuration dialog will **not persist**. Adding the password using the Registry Editor (Choose: New String Value, with the name 'PWD') will remedy this issue:

![Configuration Dialog](.\regedit_password.png)

## Add a Connection for the Snowflake database (Source)

Update and verify the Parameter Values in the EXEC statement below to correspond to your server/database specifications.

> Note: Initial Parameter Values can be substituted at runtime

In [ ]:
EXEC [elt].[Save Connection by Name] 
     @connection_name = 'snowflake-demo'
   , @server_name = 'snowflake-demo'
   , @database_name = 'CAR_CRASH'
   , @provider = 'odbc'
   , @project_name = 'Build and Execute Merge for Dimension (Snowflake)'
   , @custom_connect_string = 'DSN=snowflake-demo'

## Create Staging Schema Parameter for the Snowflake database (Source)

Verify the Parameter Values in the EXEC statement below according to your server specifications. You may need to _replace_ 'STG' with your schema name.

> Be sure to create the same schema name in your Snowflake database

In [ ]:
IF NOT EXISTS (SELECT 1 FROM [elt].[parameter] WHERE [parameter_name] = 'snowflake_staging_schema')
BEGIN
    INSERT INTO [elt].[parameter]
                ( [parameter_name]
                , [parameter_value]
                , [parameter_type] )
            VALUES
                ( 'snowflake_staging_schema'
                , 'STG'
                , 'String' )
END

## Build and Execute Merge for Dimension (Snowflake) - Project Import

The following script is used to populate the table: [ads].[project_cells] with the desired sequence numbers and file resources, along with the command used to Import this project

In [ ]:
DELETE FROM [ads].[project_cells] WHERE [project_name] = 'Build and Execute Merge for Dimension (Snowflake)'

INSERT [ads].[project_cells] ([project_name], [full_name], [cell_name], [sequence_number], [parallel_executions]) VALUES (N'Build and Execute Merge for Dimension (Snowflake)',	N'notebook_snap_basics\content\added_dim_column_names_snowflake_create.sql',	N'Create Added Dim Column Names Table (snowflake)', 100, NULL)
INSERT [ads].[project_cells] ([project_name], [full_name], [cell_name], [sequence_number], [parallel_executions]) VALUES (N'Build and Execute Merge for Dimension (Snowflake)',	N'notebook_snap_basics\content\added_dim_column_names_snowflake_insert.sql',	N'Insert into Added Dim Column Names Table (snowflake)', 110, NULL)
INSERT [ads].[project_cells] ([project_name], [full_name], [cell_name], [sequence_number], [parallel_executions]) VALUES (N'Build and Execute Merge for Dimension (Snowflake)',	N'notebook_snap_basics\content\build_and_execute_merge_for_dimension_snowflake.sql',    N'Create Dimension Merge Procedure (snowflake)', 120,	NULL)
INSERT [ads].[project_cells] ([project_name], [full_name], [cell_name], [sequence_number], [parallel_executions]) VALUES (N'Build and Execute Merge for Dimension (Snowflake)',	N'notebook_snap_basics\content\build_and_execute_merge_for_dimension_dest_fix_snowflake.sql',	N'Create Dest Dimension Self-fix Procedure (snowflake)', 130, NULL)

## Check Impact of Importing Project 

In [ ]:
EXEC [ads].[Impact Check from ADS] 'Build and Execute Merge for Dimension (Snowflake)', NULL;

## Import project from ADS

> Before running _Import Project_ step, be sure to first _Ingest_ the notebooks. See: [Ingest ADS Notebook Files (Interactive)](ingest_ads_notebook_files_interactive.ipynb)

In [ ]:
EXEC [ads].[Import Project from ADS] 'Build and Execute Merge for Dimension (Snowflake)';

## Next Step...

The associated notebook: [Build Sample Merge for Snowflake Dimension (Project Setup)](build_sample_merge_for_dimension_snowflake_project_setup.ipynb) can be used to Deploy a 'Sample Merge Statement' _Project_ into eltSnap to run against your _Snowflake_ database.